In [ ]:
# @title COCO(Common Objects in Context)

# < 특징 >
# 1. "객체 검출을 위한 대규모 데이터셋"
# 2. 다양한 객체들과 그에 대한 라벨 정보를 포함
# 3. 객체 검출, 분할, 키포인트 검출 등의 다양한 태스크에 활용
# 4. 객체 검출을 위한 bounding box 정보 뿐만 아니라 세그멘테이션 정보 등 다양한 정보를 제공
# 5. 컴퓨터 비전 분야의 연구와 알고리즘 개발에 활용

# < 형식 >
# {
#     "info": {...},        // 데이터셋 정보
#     "licenses": [...],    // 라이센스 정보
#     "images": [...],      // 이미지 정보
#     "annotations": [...], // 객체 정보 (bounding box, 클래스 등)
#     "categories": [...]    // 클래스 정보
# }

# < 형식 내용 >
# info: 데이터셋에 대한 정보를 담고 있는 딕셔너리입니다.
# licenses: 라이센스에 관한 정보를 담고 있는 리스트입니다.
# images: 이미지에 관한 정보를 담고 있는 리스트입니다.
#         각 이미지는 id, file_name, width, height 등의 정보를 포함합니다.
# annotations: 객체에 관한 정보를 담고 있는 리스트입니다.
#              객체 정보는 id, image_id, category_id, bbox (bounding box 좌표), area 등의 정보를 포함합니다.
# categories: 객체 클래스에 관한 정보를 담고 있는 리스트입니다.
#             각 클래스는 id, name, supercategory 등의 정보를 포함합니다.

In [ ]:
# @title YOLO(You Only Look Once)

# < 특징 >
# 1. "실시간 객체 검출 알고리즘"
# 2. 이미지를 한 번만 처리하여 객체를 검출하는 특징
# 3. 이미지를 그리드로 나누고, 각 그리드 셀에서 여러 개의 bounding box를 예측하여 객체를 검출
# 4. 객체를 빠르고 정확하게 검출하는데 강점

# < 형식 >
# <path_to_image> <x_center> <y_center> <width> <height> <class_id>

# < 형식 내용 >
# <path_to_image>: 이미지 파일의 경로를 나타냅니다.
# <x_center>: 객체의 bounding box의 중심 x 좌표를 이미지의 너비(width)에 대한 상대값으로 표현합니다.
# <y_center>: 객체의 bounding box의 중심 y 좌표를 이미지의 높이(height)에 대한 상대값으로 표현합니다.
# <width>: 객체의 bounding box의 너비를 이미지의 너비에 대한 상대값으로 표현합니다.
# <height>: 객체의 bounding box의 높이를 이미지의 높이에 대한 상대값으로 표현합니다.
# <class_id>: 객체의 클래스 ID를 나타냅니다.

In [ ]:
# @title (COCO -> YOLO) JSON파일을 TXT로 전환

In [46]:
# 모듈 선언
import os
import json
import glob

In [40]:
# YOLO 스타일에 TXT파일로 변환 해주는 함수

def convert_coco_to_yolo(coco_json_folder, output_txt_folder, image_folder):

    # 폴더를 생성하는 함수
    os.makedirs(output_txt_folder, exist_ok=True)
    # output_txt_folder: 폴더를 생성할 경로를 나타내는 문자열
    # exist_ok: 이 인자는 선택적 매개변수
    #           True : 폴더가 이미 존재하는 경우에는 FileExistsError 예외를 발생
    #           True : 폴더가 이미 존재해도 예외를 발생시키지 않고 무시

    # 해당 파일들의 리스트를 json_files 변수에 저장하는 코드
    json_files = glob.glob(os.path.join(coco_json_folder, '*.json'))

    for json_file in json_files:

      # JSON 파일을 텍스트 모드로 읽어옴
      with open(json_file, 'r') as f:
          coco_data = json.load(f)

      filename = os.path.splitext(os.path.basename(json_file))[0]
      output_txt_path = os.path.join(output_txt_folder, filename + '.txt')

      ## 읽어온 JSON 데이터를 그대로 텍스트 파일로 저장(본래 코드 - 참고용)
      ## with open('output.txt', 'w') as f:
      ##      f.write(json_data)

      # YOLO 형식의 텍스트 파일을 생성합니다.
      with open(output_txt_path, 'w') as f:

          # COCO 데이터의 각 어노테이션(annotation)을 처리합니다.
          for annotation in coco_data['annotations']:

              # 어노테이션 정보에서 이미지 ID, 파일명, 너비, 높이, 클래스 ID, 바운딩 박스 정보를 추출합니다.
              image_id = annotation['image_id']

              ## next(iterator[, default])
              ## -> iterator(반복자)에서 다음 요소를 반환하는 역할

              image_info = next(image for image in coco_data['images'] if image['id'] == image_id)
              # -> id 값이 image_id와 일치하는 첫 번째 이미지 정보를 가져옴

              image_filename = image_info['file_name']
              image_width = image_info['width']
              image_height = image_info['height']

              class_id = annotation['category_id']
              bbox = annotation['bbox']

              ## COCO 형식의 bounding box는 [x_min, y_min, width, height]
              ## [왼쪽 상단 모서리의 x 좌표, 왼쪽 상단 모서리의 y 좌표, 너비, 높이]

              x_center = bbox[0] + bbox[2] / 2    # x 좌표를 이미지의 너비에 대한 상대값으로 표현
              y_center = bbox[1] + bbox[3] / 2    # y 좌표를 이미지의 높이에 대한 상대값으로 표현
              width = bbox[2]                     # 이미지의 너비에 대한 상대값으로 표현
              height = bbox[3]                    # 이미지의 높이에 대한 상대값으로 표현

              # YOLO 형식에 맞게 바운딩 박스의 중심 좌표와 크기를 이미지의 크기로 정규화합니다.
              x_center /= image_width
              y_center /= image_height
              width /= image_width
              height /= image_height

              # 이미지 파일의 전체 경로를 생성합니다.
              image_path = os.path.join(image_folder, image_filename)

              ## YOLO 형식의 bounding box는 [x_center, y_center, width, height]
              # YOLO 텍스트 파일에 해당 이미지의 정보를 기록합니다.
              line = f'{image_path} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {class_id}\n'

              # 파일에 작성
              f.write(line)

In [42]:
# COCO JSON 파일 경로
coco_json_folder = '/content/drive/MyDrive/Project_3/New_Sample/라벨링데이터/TL/03/03/'

# 생성할 YOLO 텍스트 파일 경로
output_txt_folder = './txt_file_sample/'

# 이미지 파일이 저장된 폴더 경로
image_folder = '/content/drive/MyDrive/Project_3/New_Sample/원천데이터/TS/03/03/11/'

In [ ]:
# 함수 실행
convert_coco_to_yolo(coco_json_folder, output_txt_folder, image_folder)